In [38]:
import pandas as pd

In [39]:
quary = """
WITH activity_periods AS (
    SELECT
        user_id,
        date,
        log,
        ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY date) -
        ROW_NUMBER() OVER (PARTITION BY user_id, log ORDER BY date) AS period_id
    FROM
        users_log
),
activity_periods_summary AS (
    SELECT
        user_id,
        MIN(date) AS start_date,
        MAX(date) AS end_date,
        log,
        COUNT(*) AS length
    FROM
        activity_periods
    GROUP BY
        user_id,
        log,
        period_id
)
SELECT
    user_id,
    start_date,
    end_date,
    log,
    SUM(length) AS length
FROM
    activity_periods_summary
GROUP BY
    user_id,
    start_date,
    end_date,
    log
ORDER BY
    user_id,
    start_date,
    end_date;

"""

In [40]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('dkcup_test')

# Create a cursor object
cursor = conn.cursor()

# Now you can execute SQL queries using the cursor
cursor.execute(quary)

# Fetch the results
rows = cursor.fetchall()


# Close the connection
conn.close()

result_df = pd.DataFrame(rows, columns=["user_id", "start_date", "end_date", "log",	"length"]) 

In [41]:
result_df

,user_id,start_date,end_date,log,length
0,51725,2019-01-01,2019-01-01,1,1
1,51725,2019-01-02,2019-01-03,0,2
2,51725,2019-01-04,2019-01-05,1,2
3,51725,2019-01-06,2019-01-06,0,1
4,51725,2019-01-07,2019-01-10,1,4
...,...,...,...,...,...
888,973509,2019-03-22,2019-03-22,1,1
889,973509,2019-03-23,2019-03-26,0,4
890,973509,2019-03-27,2019-03-28,1,2
891,973509,2019-03-29,2019-03-30,0,2
